In [1]:
import os
import matplotlib 
# set directory
os.chdir("/nas/ucb/oliveradk/diverse-gen/")

In [2]:
from tqdm import tqdm
from pathlib import Path
from datetime import datetime
from itertools import product
import json
import copy

import numpy as np

from losses.loss_types import LossType
from utils.exp_utils import get_executor, run_experiments

In [3]:
SCRIPT_NAME = "spur_corr_exp.py"
EXP_DIR = "output/cc_dbat_aux_weight_sweep"
n_trials = 32

In [4]:
method_configs = {
    "DBAT": {
        "loss_type": LossType.DBAT, 
        "shared_backbone": False, 
        "freeze_heads": True, 
        "binary": True, 
    },
}

dataset_configs = {
    "toy_grid": {"dataset": "toy_grid", "model": "toy_model", "epochs": 100, "batch_size": 32, "target_batch_size": 128, "lr": 1e-3, "optimizer": "sgd"},
    "fmnist_mnist": {"dataset": "fmnist_mnist", "model": "Resnet50", "epochs": 5, "batch_size": 32, "target_batch_size": 64},
    "cifar_mnist": {"dataset": "cifar_mnist", "model": "Resnet50", "epochs": 5, "batch_size": 32, "target_batch_size": 64},
    "waterbirds": {"dataset": "waterbirds", "model": "Resnet50", "epochs": 5, "batch_size": 32, "target_batch_size": 64},
    "celebA-0": {"dataset": "celebA-0", "model": "Resnet50", "epochs": 2, "batch_size": 32, "target_batch_size": 64},
    # "multi-nli": {"dataset": "multi-nli", "model": "bert", "epochs": 1, "lr": 1e-5, "combine_neut_entail": True, "contra_no_neg": True},
}

method_ranges = {
    "DBAT": {"aux_weight": [-1, 1]},
}

configs = {}
for (ds_name, ds_config), (method_name, method_config) in product(dataset_configs.items(), method_configs.items()):
    configs[(ds_name, method_name)] = {**ds_config, **method_config}
    if "batch_size" in ds_config:
        configs[(ds_name, method_name)]["batch_size"] = int(ds_config["batch_size"] / 2)
        configs[(ds_name, method_name)]["target_batch_size"] = int(ds_config["target_batch_size"] / 2)



def get_conf_exp_dir(ds_name, method_name, i):
    return Path(EXP_DIR, f"{ds_name}_{method_name}/{i}")

sampled_configs = []
for (ds_name, method_name), conf in configs.items():
    for i in range(n_trials):
        sample_conf = copy.deepcopy(conf)
        sample_range = method_ranges[method_name]["aux_weight"]
        aux_weight = 10**(np.random.uniform(sample_range[0], sample_range[1]))
        seed = np.random.randint(0, 1000000)
        sample_conf["aux_weight"] = aux_weight
        sample_conf["seed"] = seed
        sample_conf["exp_dir"] = get_conf_exp_dir(ds_name, method_name, i)
        sample_conf["plot_activations"] = False
        sampled_configs.append(sample_conf)

# Run Experiments

In [13]:
low_mem_configs = [conf for conf in sampled_configs if conf["dataset"] not in ["celebA-0", "multi-nli"]]
high_mem_configs = [conf for conf in sampled_configs if conf["dataset"] in ["celebA-0", "multi-nli"]]


In [18]:
executor = get_executor(EXP_DIR, mem_gb=16)
jobs = run_experiments(executor, low_mem_configs, SCRIPT_NAME)


In [19]:
executor = get_executor(EXP_DIR, mem_gb=32)
jobs = run_experiments(executor, high_mem_configs, SCRIPT_NAME)


# Process Results

In [5]:
import yaml
import pandas as pd
all_results = []
not_found_configs = []
for (ds_name, method_name), config in configs.items():
    n_trials_ds = n_trials if not ds_name.startswith("diamonds") else n_trials // 8
    for i in range(n_trials_ds):
        exp_dir = get_conf_exp_dir(ds_name, method_name, i)
        metrics_path = Path(exp_dir, "metrics.json")
        config_path = Path(exp_dir, "config.yaml")
        if not metrics_path.exists():
            # print(f"ds_name: {ds_name}, method_name: {method_name}, i: {i}")
            not_found_configs.append((ds_name, method_name, i))
            continue
        with open(metrics_path, "r") as f:
            metrics = json.load(f)
        with open(config_path, "r") as f:
            config = yaml.safe_load(f)
        min_val_loss = min(metrics["val_loss"])
        all_results.append({
            "dataset": ds_name,
            "method_name": method_name,
            "aux_weight": config["aux_weight"],
            "lr": config["lr"],
            "loss": min_val_loss,
        })

df = pd.DataFrame(all_results, columns=["dataset", "method_name", "aux_weight", "lr", "loss"])

In [8]:
import pandas as pd
import plotly.graph_objects as go

def plot_aux_weight_vs_loss(df, dataset=None, method_name=None, use_log_scale=True):
    # Filter data based on conditions
    plot_df = df.copy()
    if dataset:
        plot_df = plot_df[plot_df['dataset'] == dataset]
    if method_name:
        plot_df = plot_df[plot_df['method_name'] == method_name]
    
    # Create figure
    fig = go.Figure()
    
    # Group by conditions that weren't filtered
    groupby_cols = []
    if not dataset:
        groupby_cols.append('dataset')
    if not method_name:
        groupby_cols.append('method_name')
    
    if groupby_cols:
        # Plot separate lines for each group
        for name, group in plot_df.groupby(groupby_cols):
            name_str = '_'.join([str(n) for n in name]) if isinstance(name, tuple) else str(name)
            # Sort by aux_weight before plotting
            group = group.sort_values('aux_weight')
            fig.add_trace(go.Scatter(
                x=group['aux_weight'],
                y=group['loss'],
                name=name_str,
                mode='markers+lines'
            ))
    else:
        # Single line plot
        # Sort by aux_weight before plotting
        print(len(plot_df))
        plot_df = plot_df.sort_values('aux_weight')
        print(len(plot_df))
        fig.add_trace(go.Scatter(
            x=plot_df['aux_weight'],
            y=plot_df['loss'],
            mode='markers+lines'
        ))
    
    # Update layout
    title = 'Aux Weight vs Loss'
    if dataset:
        title += f' for {dataset}'
    if method_name:
        title += f' with {method_name}'
    
    fig.update_layout(
        title=title,
        xaxis_title='Aux Weight',
        yaxis_title='Loss',
        width=800,
        height=500
    )
    
    if use_log_scale:
        fig.update_xaxes(type='log')
        fig.update_yaxes(type='log')
    
    return fig


32
32


In [12]:

method_name = "DBAT"

fig = plot_aux_weight_vs_loss(df, method_name=method_name, dataset="toy_grid")
fig.show()

fig = plot_aux_weight_vs_loss(df, method_name=method_name, dataset="cifar_mnist")
fig.show()

fig = plot_aux_weight_vs_loss(df, method_name=method_name, dataset="fmnist_mnist")
fig.show()

fig = plot_aux_weight_vs_loss(df, method_name=method_name, dataset="waterbirds")
fig.show()

fig = plot_aux_weight_vs_loss(df, method_name=method_name, dataset="celebA-0")
fig.show()



32
32


32
32


32
32


32
32


32
32
